# Wrangling presidential election data 

by Jae Yeon Kim 

I redid the data wrangling using Python for Python users. 

In [1]:
import datetime 
import time    

print('This is the time when this notebook was compiled:', time.strftime('%Y-%m-%d %H:%M:%S'))

This is the time when this notebook was compiled: 2020-11-12 08:38:01


In [2]:
## Load libraries 

import sys 
import numpy as np 
np.set_printoptions(threshold=sys.maxsize) # Dispaly the whole array 

import pandas as pd 
from pyprojroot import here

Huge thanks to [`tonmcg`](https://github.com/tonmcg) for putting this data together: https://github.com/tonmcg/US_County_Level_Election_Results_08-20

In [3]:
election_08_16 = pd.read_csv("https://github.com/tonmcg/US_County_Level_Election_Results_08-20/raw/master/US_County_Level_Presidential_Results_08-16.csv")
election_20 = pd.read_csv("https://github.com/tonmcg/US_County_Level_Election_Results_08-20/raw/master/2020_US_County_Level_Presidential_Results.csv")

## Wrangle data 

### Find common columns 

No common columns because of the column name inconsistency. 

In [4]:
election_08_16.columns.intersection(election_20.columns)

Index([], dtype='object')

Fix this issue by renaming the column names of `election_20`.

In [5]:
# Check column names 
election_08_16.columns

Index(['fips_code', 'county', 'total_2008', 'dem_2008', 'gop_2008', 'oth_2008',
       'total_2012', 'dem_2012', 'gop_2012', 'oth_2012', 'total_2016',
       'dem_2016', 'gop_2016', 'oth_2016'],
      dtype='object')

In [6]:
# Check column names 
election_20.columns

Index(['state_name', 'county_fips', 'county_name', 'votes_gop', 'votes_dem',
       'total_votes', 'diff', 'per_gop', 'per_dem', 'per_point_diff'],
      dtype='object')

In [5]:
# Rename column names 
election_20 = election_20.rename(columns = {'county_fips':'fips_code',
                                            'county_name':'county'})

In [6]:
# Check the common columns
election_08_16.columns.intersection(election_20.columns)

Index(['fips_code', 'county'], dtype='object')

In [7]:
common_column = election_08_16.columns.intersection(election_20.columns)

In [8]:
print(f'The name of the common column is {common_column.values}.')

The name of the common column is ['fips_code' 'county'].


## Join the two datasets 

Caveat 1: Note that we have three more counties (including a NUll value) in the `election_20` dataframe than the `election_08_16` dataframe.

In [9]:
election_08_16.county.nunique()

1845

In [37]:
election_20.county.nunique()

1847

The following is not true because electioin_20.county includes a NULL value. 

In [44]:
len(election_20.county.unique()) == election_20.county.nunique() 

False

In [45]:
len(election_20.county.unique()) - election_20.county.nunique() 

1

Caveat 2: Also, 64 county names in the two dataframes didn't match.

In [108]:
# Differently named counties 
list_out = [value for value in list(election_08_16.county) if value not in list(election_20.county)]

In [109]:
# Remove duplicates 
len(set(list_out))

64

In [110]:
# Or you can use set directly to calculte the length of the uniquely named counties 
len(set(election_08_16.county).difference(list(election_20.county)))

64

- Change the column names of the `election_20` in the same style of `election_08_16` dataframe. 

In [15]:
election_20 = election_20.rename(
    columns = {"votes_dem" : "dem_2020",
              "votes_gop" : "gop_2020",
              "total_votes" : "total_2020"})

In [17]:
election_20 = election_20.drop(election_20.filter(regex = 'per_|diff'), axis = 1)

Since we're trying to create a panel data, I join the two datasets (x1, x2) based on x1.

In [16]:
joined = pd.merge(election_08_16, election_20, how = 'left')


## Select

Let's focus on the 2016 and 2020 presidential election results. 

In [17]:
selected = joined.filter(regex = '2016|2020|fips_code|county|state_name')

## Mutate 

Calculate party vote share in each election and add that information to the original dataframe.

In [18]:
dem_diff = round((selected.dem_2020/selected.total_2020) - (selected.dem_2016/selected.total_2016), 4)
gop_diff = round((selected.gop_2020/selected.total_2020) - (selected.gop_2016/selected.total_2016), 4)

In [19]:
augmented = selected.copy()

In [20]:
augmented['dem_diff'] = dem_diff
augmented['gop_diff'] = gop_diff

In [21]:
augmented.head()

,fips_code,county,total_2016,dem_2016,gop_2016,oth_2016,state_name,gop_2020,dem_2020,total_2020,dem_diff,gop_diff
0,26041,Delta County,18467,6431,11112,924,Michigan,13206.0,7605.0,21154.0,0.0113,0.0226
1,48295,Lipscomb County,1322,135,1159,28,Texas,1203.0,131.0,1351.0,-0.0052,0.0137
2,1127,Walker County,29243,4486,24208,549,Alabama,25947.0,4826.0,31106.0,0.0017,0.0063
3,48389,Reeves County,3184,1659,1417,108,Texas,2249.0,1394.0,3683.0,-0.1425,0.1656
4,56017,Hot Springs County,2535,400,1939,196,Wyoming,1999.0,482.0,2577.0,0.0292,0.0108


In [24]:
augmented.shape

(3112, 12)

## Export 

In [24]:
# Check the file path 
here('outputs/wrangled_python_updated.csv')

PosixPath('/home/jae/us_election_2020/outputs/wrangled_python_updated.csv')

In [25]:
# Save the file 
# augmented.to_csv(here('outputs/wrangled.csv'))